In [1]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict

In [2]:
class Batsman(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float
    bpb: float
    boundary_percentage: float
    summary: str

In [3]:
def calculate_sr(state: Batsman) -> Batsman:
    sr = (state['runs'] / state['balls']) * 100
    return {'sr':sr}
def calculate_bpb(state: Batsman) -> Batsman:
    bpb = state['balls'] / state['runs'] if state['runs'] != 0 else 0
    return {'bpb':bpb}
def calculate_boundary_percentage(state: Batsman) -> Batsman:
    total_boundaries = state['fours'] + state['sixes']
    boundary_percentage = (total_boundaries / state['balls']) * 100 if state['balls'] != 0 else 0
    return {'boundary_percentage':boundary_percentage}

In [4]:
def summary(state: Batsman) -> str:
    return {'summary':f"Batsman scored {state['runs']} runs off {state['balls']} balls with a strike rate of {state['sr']:.2f}, balls per run of {state['bpb']:.2f}, and boundary percentage of {state['boundary_percentage']:.2f}%."}

In [5]:
graph=StateGraph(Batsman)
graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percentage',calculate_boundary_percentage)
graph.add_node('summary',summary)

graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundary_percentage')
graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percentage','summary')
graph.add_edge('summary',END)   
workflow=graph.compile()

In [6]:
workflow.invoke({
    'runs': 120,
    'balls': 75,
    'fours': 10,
    'sixes': 5})

{'runs': 120,
 'balls': 75,
 'fours': 10,
 'sixes': 5,
 'sr': 160.0,
 'bpb': 0.625,
 'boundary_percentage': 20.0,
 'summary': 'Batsman scored 120 runs off 75 balls with a strike rate of 160.00, balls per run of 0.62, and boundary percentage of 20.00%.'}

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
)

In [10]:
model.invoke("what is name of india's capital?")

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b6e77-31b2-7dc0-a769-d7292db2bfb6-0', usage_metadata={'input_tokens': 10, 'output_tokens': 37, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 28}})

In [11]:
from pydantic import BaseModel, Field

In [12]:
class EvaluationSchema(BaseModel):
   feedback: str = Field(..., description="Feedback on the model's response")
   score: int = Field(..., ge=1, le=10, description="Score from 1 to 10 evaluating the response quality")
   

In [13]:
structured_model=model.with_structured_output(EvaluationSchema)

In [15]:
x=model.invoke("Explain the theory of relativity in simple terms.").content

In [16]:
x

'The theory of relativity, developed by Albert Einstein, fundamentally changed our understanding of space, time, gravity, and the universe. It\'s often broken down into two main parts: **Special Relativity** and **General Relativity**.\n\nLet\'s break them down in simple terms:\n\n---\n\n### The Core Idea (The "Relativity" Part)\n\nImagine you\'re on a train moving smoothly. If you drop a ball, it falls straight down. Someone standing on the ground watching the train go by would see the ball move in an arc as it falls, because they\'re also seeing the train (and you) move forward.\n\nThe *events* (dropping the ball, the ball hitting the floor) are the same, but how different observers *measure* those events (the path the ball takes) depends on their relative motion. This is the essence of "relativity" – there\'s no single, absolute viewpoint from which to measure everything.\n\n### Part 1: Special Relativity (1905)\n\nThis part deals with how space and time are relative for observers i

In [17]:
prompt="evaluate the language quality of following essay and provide a feedback along with a score from 1 to 10:\n\n"+x
structured_model.invoke(prompt)


EvaluationSchema(feedback='The language quality of this essay is exceptionally clear, precise, and highly effective in explaining complex scientific concepts to a general audience. The use of simple, accessible terms and well-chosen analogies (like the train and the rubber sheet) is outstanding, making challenging ideas like time dilation and spacetime curvature remarkably easy to grasp. The essay maintains an engaging and conversational tone throughout, which significantly aids comprehension. The structure is logical with clear headings, and the flow between sections is smooth. There are no grammatical errors or awkward phrasing, demonstrating excellent command of written English. It successfully balances scientific accuracy with pedagogical clarity.', score=9)

In [21]:
from typing import TypedDict, Annotated, List
import operator

class UpscState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[List[int], operator.add]
    avg_score: float

In [22]:
def evaluate_language(state: UpscState):
  prompt = f"""Evaluate the language quality of the following essay and provide feedback along with a score from 1 to 10:{state['essay']}"""
  output=structured_model.invoke(prompt)
  return {'language_feedback':output.feedback,'individual_scores':[output.score]}
def evaluate_analysis(state: UpscState):
  prompt = f"""Evaluate the analysis quality of the following essay and provide feedback along with a score from 1 to 10:{state['essay']}"""
  output=structured_model.invoke(prompt)
  return {'analysis_feedback':output.feedback,'individual_scores':[output.score]}
def evaluate_clarity(state: UpscState):
  prompt = f"""Evaluate the clarity of the following essay and provide feedback along with a score from 1 to 10:{state['essay']}"""
  output=structured_model.invoke(prompt)
  return {'clarity_feedback':output.feedback,'individual_scores':[output.score]}
def evaluate_overall(state: UpscState):
  prompt = f"""Evaluate the overall quality of the following essay and provide feedback along with a score from 1 to 10:{state['essay']}"""
  output=structured_model.invoke(prompt)
  avg_score=sum(state['individual_scores'])/len(state['individual_scores'])
  return {'overall_feedback':output.feedback,'avg_score':avg_score}



In [23]:
graph=StateGraph(UpscState)
graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_clarity',evaluate_clarity)
graph.add_node('evaluate_overall',evaluate_overall)
graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_clarity')
graph.add_edge('evaluate_language','evaluate_overall')
graph.add_edge('evaluate_analysis','evaluate_overall')
graph.add_edge('evaluate_clarity','evaluate_overall')
graph.add_edge('evaluate_overall',END)   
workflow=graph.compile()

In [24]:
workflow.invoke({
    'essay': x,
    })

{'essay': 'The theory of relativity, developed by Albert Einstein, fundamentally changed our understanding of space, time, gravity, and the universe. It\'s often broken down into two main parts: **Special Relativity** and **General Relativity**.\n\nLet\'s break them down in simple terms:\n\n---\n\n### The Core Idea (The "Relativity" Part)\n\nImagine you\'re on a train moving smoothly. If you drop a ball, it falls straight down. Someone standing on the ground watching the train go by would see the ball move in an arc as it falls, because they\'re also seeing the train (and you) move forward.\n\nThe *events* (dropping the ball, the ball hitting the floor) are the same, but how different observers *measure* those events (the path the ball takes) depends on their relative motion. This is the essence of "relativity" – there\'s no single, absolute viewpoint from which to measure everything.\n\n### Part 1: Special Relativity (1905)\n\nThis part deals with how space and time are relative for o